# **Machine Unlearning: el arte de olvidar en la era de la Inteligencia Artificial**


### Descripción del Notebook

En este notebook se proporciona el código para entrenar tres modelos diferentes como parte del Trabajo de Fin de Grado (TFG) titulado "Machine Unlearning: el arte de olvidar en la era de la Inteligencia Artificial", realizado por Pablo Noriega Vázquez:

1. **Modelo Original**:
   - Utiliza un modelo preentrenado de ResNet50 para realizar una tarea de regresión. Se carga un conjunto de datos original y se entrena el modelo utilizando el conjunto de datos sin modificaciones.

2. **Modelo de Fine-Tuning Básico**:
   - Carga el modelo original y realiza un fine-tuning borrando un rango de edades con la finalidad de aplicar un proceso de unlearning.

3. **Modelo de Fine-Tuning con Etiquetas Aleatorias**:
   - En este modelo se simula un escenario de fine-tuning más desafiante. Se carga un conjunto de datos original y se realiza un fine-tuning con etiquetas aleatorias. Esto implica una modificación selectiva de las edades en los datos de entrenamiento para mejorar la robustez del modelo.


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from zipfile import ZipFile
import h5py
import pickle

### En esta celda de código se realiza el siguiente proceso:

1. **Montar Google Drive**:
   - Se conecta Google Drive en el entorno de ejecución de Google Colab. Esto permite acceder y manipular los archivos almacenados en Google Drive desde el entorno de Colab.

2. **Instalar gdown**:
   - Se instala la herramienta `gdown` para descargar archivos desde Google Drive directamente desde Colab.

3. **Descargar y descomprimir el archivo desde Google Drive**:
   - Se define el enlace del archivo en Google Drive a través de su ID.
   - Se utiliza `gdown` para descargar el archivo desde Google Drive a Colab.
   - Luego, se descomprime el archivo descargado.

4. **Eliminar el archivo .zip después de la extracción**:
   - Una vez que el archivo se ha descomprimido con éxito, se elimina el archivo `.zip` para liberar espacio en el entorno de Colab.

Este proceso permite descargar y descomprimir el archivo en el que incluimos modelos y bases de datos, almacenado en Google Drive en el entorno de ejecución de Google Colab.


In [ ]:
# Montar Google Drive
drive.mount('/content/gdrive', force_remount=True)

# Descargar y descomprimir el archivo desde Google Drive
!pip install gdown

# Enlace de Google Drive
file_id = '1HtTUZnnXz5dVAPLyDo8Vhj6IyFYfxL0-'
gdown_url = f'https://drive.google.com/uc?id={file_id}'

# Descargar el archivo
!gdown $gdown_url -O data.zip

# Descomprimir el archivo
with ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall()
    print('Model decompressed successfully')

# Eliminar el archivo .zip después de la extracción para liberar espacio
os.remove('data.zip')


# **ENTRENAR MODELO ORIGINAL PREDICCIÓN DE EDAD**

1. **Cargar los datos**: Se cargan los datos de entrenamiento, validación y prueba desde archivos `.npy` ubicados en el directorio `./NoriegaVazquezPablo_TFG_CODE/originalData`.

2. **Normalizar etiquetas**: Se normalizan las etiquetas dividiéndolas por 60. Esto se aplica a las etiquetas de entrenamiento (`Y_train`), validación (`Y_valid`) y prueba (`Y_test`).

3. **Verificar dimensiones**: Se imprimen las dimensiones de los datos cargados para asegurar que todo se ha cargado correctamente y las dimensiones son las esperadas.


In [ ]:
# Cargar los conjuntos de datos
X_train = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'X_train.npy'))
Y_train = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'Y_train.npy'))
Y_train = Y_train / 60  # Normalizar etiquetas de entrenamiento

X_valid = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'X_valid.npy'))
Y_valid = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'Y_valid.npy'))
Y_valid = Y_valid / 60  # Normalizar etiquetas de validación

X_test = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'X_test.npy'))
Y_test = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'Y_test.npy'))
Y_test = Y_test / 60  # Normalizar etiquetas de prueba

# Imprimir dimensiones de los conjuntos de datos
print("Dimensiones de los conjuntos de entrenamiento:")
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)

print("\nDimensiones de los conjuntos de validación:")
print("X_valid:", X_valid.shape)
print("Y_valid:", Y_valid.shape)

print("\nDimensiones de los conjuntos de test:")
print("X_test:", X_test.shape)
print("Y_test:", Y_test.shape)


**Preprocesar conjunto de entrenamiento, validación y prueba**:
   - Se convierten los datos a tipo `float32` para asegurar que están en el formato correcto para la red neuronal.
   - Para cada imagen, se expande una dimensión adicional para que tenga la forma esperada por la función `preprocess_input` de ResNet50.
   - Luego, se aplica `preprocess_input` de ResNet50 para normalizar las imágenes de acuerdo con los requisitos del modelo preentrenado.

Esta preprocesamiento es necesario porque ResNet50, como muchas redes neuronales preentrenadas, espera que las imágenes de entrada estén normalizadas de una manera específica, lo cual ayuda a mejorar la precisión y la convergencia del modelo.


In [5]:
# train set
X_train = X_train.astype('float32')
for i in range(0,X_train.shape[0]):
  x = X_train[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_train[i,] = tf.keras.applications.resnet50.preprocess_input(x)

# validation set
X_valid = X_valid.astype('float32')
for i in range(0,X_valid.shape[0]):
  x = X_valid[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_valid[i,] = tf.keras.applications.resnet50.preprocess_input(x)

X_test = X_test.astype('float32')
for i in range(0,X_test.shape[0]):
  x = X_test[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_test[i,] = tf.keras.applications.resnet50.preprocess_input(x)

En esta celda de código, se realiza lo siguiente:

1. **Descarga y descompresión de datos**:
   - Se descarga un archivo comprimido que contiene el modelo preentrenado.
   - Luego, se descomprime el archivo para obtener el modelo.

2. **Carga del modelo preentrenado**:
   - Se carga el modelo preentrenado desde el archivo `weights.h5`.

3. **Modificación y ampliación del modelo**:
   - Se agregan capas de dropout para evitar el sobreajuste.
   - Se añaden capas Fully Connected (FC) ocultas para aprender representaciones ocultas.
   - Se configura una capa de salida para un problema de regresión.

4. **Construcción del modelo final**:
   - Se construye el modelo final que incluye las modificaciones realizadas.


In [ ]:
# Descargar los datos
!wget http://data.chalearnlap.cvc.uab.cat/Colab_MFPDS/model.zip

# Descomprimir los datos
with ZipFile('model.zip', 'r') as zip:
   zip.extractall()
   print('Modelo descomprimido exitosamente')

# Eliminar el archivo .zip después de la extracción para liberar espacio
!rm model.zip

# Cargar el modelo preentrenado
model = tf.keras.models.load_model('./model/weights.h5')

# Utilizar la capa FC antes de la capa 'classifier_low_dim' como vector de características
fc_512 = model.get_layer('dim_proj').output

# Agregar una capa de dropout para minimizar problemas de sobreajuste
dp_layer = tf.keras.layers.Dropout(0.5)(fc_512)

# Agregar algunas capas FC ocultas para aprender representaciones ocultas
fc_256 = tf.keras.layers.Dense(256, activation='relu', name='f_256')(dp_layer)

# Agregar otra capa de dropout para minimizar problemas de sobreajuste
dp_layer2 = tf.keras.layers.Dropout(0.2)(fc_256)
fc_128 = tf.keras.layers.Dense(128, activation='relu', name='f_128')(dp_layer2)

# Para regresión, típicamente usamos un solo neurona en la capa de salida con activación lineal
output_aux = tf.keras.layers.Dense(1, name='output')(fc_128)
output = tf.keras.layers.Activation('sigmoid', name='predict')(output_aux)

# Construir e imprimir el modelo final
model = tf.keras.models.Model(inputs=model.get_layer('base_input').output, outputs=output)


En esta celda de código, se realiza lo siguiente:

1. **Configuración de parámetros**:
   - Se definen los parámetros para el entrenamiento del modelo, como el número de épocas (`NUM_EPOCHS`), la tasa de aprendizaje (`lr`) y el tamaño del lote (`batch_s`).

2. **Ruta de guardado del modelo**:
   - Se define la ruta donde se guardará el modelo entrenado (`model_name`).

3. **Definición de generadores de datos**:
   - Se utilizan generadores de datos para el conjunto de entrenamiento y validación, lo que permite aplicar el aumento de datos de forma dinámica durante el entrenamiento.

4. **Compilación del modelo**:
   - Se compila el modelo utilizando el optimizador Adam y la función de pérdida de error absoluto medio (`mean_absolute_error`).

5. **Definición de callbacks**:
   - Se definen callbacks, como `EarlyStopping` para detener el entrenamiento prematuramente si la pérdida en el conjunto de validación deja de disminuir, y `ModelCheckpoint` para guardar el mejor modelo obtenido durante el entrenamiento.

6. **Entrenamiento del modelo**:
   - Se entrena el modelo utilizando los generadores de datos de entrenamiento y validación, así como los parámetros definidos anteriormente y los callbacks especificados.

7. **Guardado del historial de entrenamiento**:
    - Se guarda el historial de entrenamiento del modelo en un archivo para su posterior análisis.


In [ ]:
# Número de epochs de entrenamiento
NUM_EPOCHS = 120

# Tasa de aprendizaje
lr = 1e-5

# Tamaño del lote (batch size)
batch_s = 32

#Ruta donde se guardará el modelo
model_name = '/content/gdrive/MyDrive/original_model.h5'

# Definir parámetros de aumento de datos
datagen = ImageDataGenerator()

# Convertir las etiquetas one-hot a etiquetas de clase
# y_train_classes = np.argmax(Y_train_onehot, axis=1)

# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mean_absolute_error')

# Definir el generador para los datos de entrenamiento con aumento de datos
train_generator = datagen.flow(X_train, Y_train, batch_size=batch_s)
valid_generator = datagen.flow(X_valid, Y_valid, batch_size=batch_s)

# Definir los callbacks
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
mc = tf.keras.callbacks.ModelCheckpoint(model_name, monitor='val_loss', mode='min', save_best_only=True)

# Entrenar el modelo
history = model.fit(train_generator, validation_data=valid_generator, batch_size=batch_s, epochs=NUM_EPOCHS, shuffle=True, verbose=1, callbacks=[es, mc])

# Guardar el historial de entrenamiento
with open(model_name, 'wb') as handle:
    pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)


# **ENTRENAR MODELO UNLEARNING 1 (FINE-TUNING BÁSICO)**

1. **Cargar los datos**: Se cargan los datos de entrenamiento, validación y prueba desde archivos `.npy` ubicados en el directorio `./NoriegaVazquezPablo_TFG_CODE/dataWithout_20-28`. En este conjunto se han borrado las muestras entre 20 y 28 años.

2. **Normalizar etiquetas**: Se normalizan las etiquetas dividiéndolas por 60. Esto se aplica a las etiquetas de entrenamiento (`Y_train`), validación (`Y_valid`) y prueba (`Y_test`).

3. **Verificar dimensiones**: Se imprimen las dimensiones de los datos cargados para asegurar que todo se ha cargado correctamente y las dimensiones son las esperadas.

In [ ]:
# Cargar los conjuntos de datos

# Cargar el conjunto de entrenamiento
X_train = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/dataWithout_20-28', 'X_train_without_20-28.npy'))
Y_train = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/dataWithout_20-28', 'Y_train_without_20-28.npy'))
Y_train = Y_train / 60  # Normalizar etiquetas de entrenamiento

# Cargar el conjunto de validación
X_valid = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/dataWithout_20-28', 'X_valid_without_20-28.npy'))
Y_valid = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/dataWithout_20-28', 'Y_valid_without_20-28.npy'))
Y_valid = Y_valid / 60  # Normalizar etiquetas de entrenamiento

# Cargar el conjunto de prueba
X_test = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/dataWithout_20-28', 'X_test_without_20-28.npy'))
Y_test = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/dataWithout_20-28', 'Y_test_without_20-28.npy'))
Y_test = Y_test / 60  # Normalizar etiquetas de entrenamiento

# Imprimir las dimensiones de los conjuntos de datos
print("Dimensiones de los conjuntos de entrenamiento:")
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)

print("\nDimensiones de los conjuntos de validación:")
print("X_valid:", X_valid.shape)
print("Y_valid:", Y_valid.shape)

print("\nDimensiones de los conjuntos de test:")
print("X_test:", X_test.shape)
print("Y_test:", Y_test.shape)

**Preprocesar conjunto de entrenamiento, validación y prueba**:
   - Se convierten los datos a tipo `float32` para asegurar que están en el formato correcto para la red neuronal.
   - Para cada imagen, se expande una dimensión adicional para que tenga la forma esperada por la función `preprocess_input` de ResNet50.
   - Luego, se aplica `preprocess_input` de ResNet50 para normalizar las imágenes de acuerdo con los requisitos del modelo preentrenado.

Esta preprocesamiento es necesario porque ResNet50, como muchas redes neuronales preentrenadas, espera que las imágenes de entrada estén normalizadas de una manera específica, lo cual ayuda a mejorar la precisión y la convergencia del modelo.


In [4]:
# train set
X_train = X_train.astype('float32')
for i in range(0,X_train.shape[0]):
  x = X_train[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_train[i,] = tf.keras.applications.resnet50.preprocess_input(x)

# validation set
X_valid = X_valid.astype('float32')
for i in range(0,X_valid.shape[0]):
  x = X_valid[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_valid[i,] = tf.keras.applications.resnet50.preprocess_input(x)

X_test = X_test.astype('float32')
for i in range(0,X_test.shape[0]):
  x = X_test[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_test[i,] = tf.keras.applications.resnet50.preprocess_input(x)

###En esta celda de código, se llevan a cabo las siguientes acciones:

1. **Definición de hiperparámetros**:
   - Se define el número de epochs de entrenamiento (`NUM_EPOCHS`), la tasa de aprendizaje (`lr`), y el tamaño del lote (`batch_s`).

2. **Cargar el modelo preentrenado**:
   - Se carga el modelo preentrenado desde la ubicación especificada.

3. **Definir parámetros de aumento de datos**:
   - Se utilizan los parámetros de aumento de datos definidos mediante `ImageDataGenerator()`.

4. **Compilar el modelo**:
   - Se compila el modelo utilizando el optimizador Adam con la tasa de aprendizaje especificada y la función de pérdida de error absoluto medio (`mean_absolute_error`).

5. **Definir el generador para los datos de entrenamiento y validación**:
   - Se definen los generadores de datos para los conjuntos de entrenamiento y validación utilizando los parámetros de aumento de datos y el tamaño del lote especificados.

6. **Definir los callbacks**:
   - Se establecen los callbacks, incluyendo `EarlyStopping` para detener el entrenamiento si la pérdida en el conjunto de validación deja de disminuir, y `ModelCheckpoint` para guardar el mejor modelo obtenido durante el entrenamiento.

7. **Entrenar el modelo**:
   - Se entrena el modelo utilizando los generadores de datos de entrenamiento y validación, el número de épocas especificado y los callbacks definidos.

8. **Guardar el historial de entrenamiento**:
   - Se guarda el historial de entrenamiento en un archivo.

9. **Guardar el modelo final**:
   - Se guarda el modelo final en un archivo con el nombre especificado.


In [ ]:


# Definir los parámetros de entrenamiento
NUM_EPOCHS = 8  # Número de épocas de entrenamiento
lr = 1e-5 # Tasa de aprendizaje
batch_s = 32 # Tamaño del lote (batch size)
final_model_name = '/content/gdrive/MyDrive/temp/unlearning_fine-tuning-basic_20-60.h5' # Ruta donde se guardará el modelo


# Cargar el modelo preentrenado
model =  tf.keras.models.load_model('./NoriegaVazquezPablo_TFG_CODE/models/originalModel.h5')

# Definir parámetros de aumento de datos
datagen = ImageDataGenerator()

# Convertir las etiquetas one-hot a etiquetas de clase
# y_train_classes = np.argmax(Y_train_onehot, axis=1)

# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mean_absolute_error')

# Definir el generador para los datos de entrenamiento con aumento de datos
train_generator = datagen.flow(X_train, Y_train, batch_size=batch_s)
valid_generator = datagen.flow(X_valid, Y_valid, batch_size=batch_s)

# Definir los callbacks
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
mc = tf.keras.callbacks.ModelCheckpoint(final_model_name, monitor='val_loss', mode='min', save_best_only=True)

# Entrenar el modelo
history = model.fit(train_generator, validation_data=valid_generator, batch_size=batch_s, epochs=NUM_EPOCHS, shuffle=True, verbose=1, callbacks=[es, mc])

# Guardar el historial de entrenamiento
with open(final_model_name.replace('.h5', ''), 'wb') as handle:
    pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Guardar el modelo final
model.save(final_model_name.replace('.h5', '_final.h5'))


# **ENTRENAR MODELO UNLEARNING 2 (FINE-TUNING CON ETIQUETAS RANDOM)**

1. **Cargar los datos**: Se cargan los datos de entrenamiento, validación y prueba desde archivos `.npy` ubicados en el directorio `./NoriegaVazquezPablo_TFG_CODE/originalData`. En este conjunto se han borrado las muestras entre 20 y 28 años.

2. **Normalizar etiquetas**: Se normalizan las etiquetas dividiéndolas por 60. Esto se aplica a las etiquetas de entrenamiento (`Y_train`), validación (`Y_valid`) y prueba (`Y_test`).

3. **Verificar dimensiones**: Se imprimen las dimensiones de los datos cargados para asegurar que todo se ha cargado correctamente y las dimensiones son las esperadas.

In [ ]:
# Cargar los conjuntos de datos
X_train = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'X_train.npy'))
Y_train = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'Y_train.npy'))
Y_train = Y_train / 60  # Normalizar etiquetas de entrenamiento

X_valid = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'X_valid.npy'))
Y_valid = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'Y_valid.npy'))
Y_valid = Y_valid / 60  # Normalizar etiquetas de validación

X_test = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'X_test.npy'))
Y_test = np.load(os.path.join('./NoriegaVazquezPablo_TFG_CODE/originalData', 'Y_test.npy'))
Y_test = Y_test / 60  # Normalizar etiquetas de prueba

# Definir los límites de edad
edad_min = 20
edad_max = 28

# Encontrar índices donde la edad está dentro del rango deseado
indices_train = np.where((Y_train>= edad_min) & (Y_train <= edad_max))[0]
indices_valid = np.where((Y_valid >= edad_min) & (Y_valid<= edad_max))[0]
indices_test = np.where((Y_test >= edad_min) & (Y_test <= edad_max))[0]

Y_train_regression = Y_train / 60
Y_valid_regression = Y_valid/60
Y_test_regression = Y_test/60

# Imprimir el número de muestras encontradas en cada conjunto
print("Número de muestras en el rango de edad (20-28 años):")
print("Conjunto de entrenamiento:", len(indices_train))

print("Dimensiones de los conjuntos de entrenamiento:")
print("X_train:", X_train.shape)
print("Y_train_onehot:", Y_train_regression.shape)

print("\nDimensiones de los conjuntos de validación:")
print("X_valid:", X_valid.shape)
print("Y_valid_onehot:", Y_valid_regression.shape)

print("\nDimensiones de los conjuntos de test:")
print("X_test:", X_test.shape)
print("Y_test_onehot:", Y_test_regression.shape)

**Preprocesar conjunto de entrenamiento, validación y prueba**:
   - Se convierten los datos a tipo `float32` para asegurar que están en el formato correcto para la red neuronal.
   - Para cada imagen, se expande una dimensión adicional para que tenga la forma esperada por la función `preprocess_input` de ResNet50.
   - Luego, se aplica `preprocess_input` de ResNet50 para normalizar las imágenes de acuerdo con los requisitos del modelo preentrenado.

Esta preprocesamiento es necesario porque ResNet50, como muchas redes neuronales preentrenadas, espera que las imágenes de entrada estén normalizadas de una manera específica, lo cual ayuda a mejorar la precisión y la convergencia del modelo.


In [4]:
# train set
X_train = X_train.astype('float32')
for i in range(0,X_train.shape[0]):
  x = X_train[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_train[i,] = tf.keras.applications.resnet50.preprocess_input(x)

# validation set
X_valid = X_valid.astype('float32')
for i in range(0,X_valid.shape[0]):
  x = X_valid[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_valid[i,] = tf.keras.applications.resnet50.preprocess_input(x)

X_test = X_test.astype('float32')
for i in range(0,X_test.shape[0]):
  x = X_test[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_test[i,] = tf.keras.applications.resnet50.preprocess_input(x)

###En esta celda de código, se llevan a cabo las siguientes acciones:

1. **Cargar el modelo preentrenado**:
   - Se carga un modelo preentrenado desde la ubicación especificada en el disco.

2. **Definir los parámetros de entrenamiento**:
   - Se establecen los parámetros necesarios para el entrenamiento del modelo, como el número de epochs (`NUM_EPOCHS`), la tasa de aprendizaje (`lr`), y el tamaño del lote (`batch_s`).

3. **Definir el generador de datos de aumento** (`ImageDataGenerator`):
   - Se utiliza `ImageDataGenerator()` para definir los parámetros de aumento de datos. Esta función se utiliza para aumentar la cantidad de datos de entrenamiento mediante la aplicación de transformaciones aleatorias a las imágenes.

4. **Compilar el modelo**:
   - Se compila el modelo utilizando el optimizador Adam con la tasa de aprendizaje especificada (`lr`) y la función de pérdida de error absoluto medio (`mean_absolute_error`).

5. **Definir la función `custom_generator`**:
   - Se define una función generadora personalizada (`custom_generator`) que se utilizará para generar lotes de datos de entrenamiento con modificaciones selectivas en las edades. Esta función toma como entrada los datos de entrada (`X`) y las edades de (`Y`), el tamaño del lote (`batch_size`) y una lista de índices (`index_indices`) que indica en qué índices se deben realizar las modificaciones de las edades.
   - La función generadora baraja aleatoriamente los índices en cada epoch y luego crea lotes de datos de tamaño `batch_size`. Para los índices especificados en `index_indices`, la función modifica las edades sumándoles un número aleatorio fuera del rango [-2, -1, 0, 1, 2], asegurándose de que la nueva edad esté dentro del rango [20, 60].
   - Esto permite simular el escenario en el que se desea introducir un cierto nivel de ruido en los datos de entrenamiento para mejorar la robustez del modelo.

6. **Definir los generadores de datos**:
   - Se define un generador de datos personalizado para el entrenamiento utilizando la función `custom_generator` y otro generador de datos estándar para la validación. Estos generadores se utilizarán para proporcionar lotes de datos al modelo durante el entrenamiento y la validación.

7. **Definir callbacks**:
   - Se establecen callbacks para el entrenamiento del modelo. En este caso, se utiliza `EarlyStopping` para detener el entrenamiento prematuramente si la pérdida en el conjunto de validación deja de disminuir, y `ModelCheckpoint` para guardar el mejor modelo obtenido durante el entrenamiento.

8. **Entrenar el modelo**:
   - Se entrena el modelo utilizando el generador de datos personalizado para el entrenamiento y el generador de datos estándar para la validación. Se especifica el número de épocas (`NUM_EPOCHS`), el número de pasos por época (`steps_per_epoch`), y se utilizan los callbacks definidos anteriormente.

9. **Guardar el historial de entrenamiento**:
   - Se guarda el historial de entrenamiento del modelo en un archivo utilizando la biblioteca `pickle`. Esto permite realizar un seguimiento del progreso del entrenamiento y analizar las métricas de rendimiento a lo largo del tiempo.

10. **Guardar el modelo final**:
    - Se guarda el modelo final entrenado en un archivo con el nombre especificado. Este modelo puede ser utilizado posteriormente para hacer predicciones sobre nuevos datos.


In [ ]:
# Definir los parámetros de entrenamiento
NUM_EPOCHS = 8  # Número de épocas de entrenamiento
lr = 1e-5  # Tasa de aprendizaje
batch_s = 32  # Tamaño del lote (batch size)
final_model_name = '/content/gdrive/MyDrive/temp/unlearning_sum_36-44_16.h5' #nombre final del modelo

# Cargar el modelo preentrenado
model =  tf.keras.models.load_model('./NoriegaVazquezPablo_TFG_CODE/models/originalModel.h5')


# Definir el generador de datos de aumento (ImageDataGenerator)
datagen = ImageDataGenerator()

# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mean_absolute_error')

def custom_generator(X, Y, batch_size, index_indices):
    num_samples = len(X)
    indices = np.arange(num_samples)

    while True:
        np.random.shuffle(indices)  # Barajar aleatoriamente los índices en cada epoch

        for start in range(0, num_samples, batch_size):
            batch_indices = indices[start:start+batch_size]
            x_batch = X[batch_indices]
            y_batch = Y[batch_indices].copy()  # Copiar las edades para modificarlas

            # Modificar las edades selectivamente en los índices deseados antes de asignar al lote
            for i, idx in enumerate(batch_indices):
                if idx in index_indices:
                    # Generar un número aleatorio fuera de [-2, -1, 0, 1, 2]
                    random_offset = np.random.randint(-6, 7)
                    while random_offset in [-2, -1, 0, 1, 2]:  # Verificar si está en la lista prohibida
                        random_offset = np.random.randint(-6, 7)  # Volver a generar si es necesario
                    # Sumar este número aleatorio al valor original de la edad
                    new_age = y_batch[i] + random_offset
                    # Asegurar que la nueva edad esté dentro del rango [20, 60]
                    y_batch[i] = np.clip(new_age, 20, 60)

            yield x_batch, y_batch

# Definir el generador de datos para el entrenamiento con modificación selectiva de edades
train_generator_custom = custom_generator(X_train, Y_train_regression, batch_s, indices_train)

# Definir el generador de datos de validación estándar
valid_generator_standard = datagen.flow(X_valid, Y_valid_regression, batch_size=batch_s)

# Definir callbacks (detener temprano y guardar el mejor modelo)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
mc = tf.keras.callbacks.ModelCheckpoint(final_model_name, monitor='val_loss', mode='min', save_best_only=True)

# Entrenar el modelo utilizando el generador de datos personalizado y el generador de datos de validación estándar
history = model.fit(train_generator_custom, validation_data=valid_generator_standard, epochs=NUM_EPOCHS,
                    steps_per_epoch=len(X_train) // batch_s, shuffle=True, verbose=1, callbacks=[es, mc])

# Guardar el historial de entrenamiento
with open(final_model_name.replace('.h5', ''), 'wb') as handle:
    pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
